In [2]:
import pickle
import nltk
import tensorflow as tf
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pandas as pd
import re
from keras_preprocessing.sequence import pad_sequences
import os
!pip install gradio
import gradio as gr

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 4.9 MB/s 
     |████████████████████████████████| 2.3 MB 41.3 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 213 kB 61.5 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 272 kB 52.2 MB/s 
     |████████████████████████████████| 106 kB 54.9 MB/s 
     |████████████████████████████████| 54 kB 1.1 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 64 kB 1.5 MB/s 
     |████████████████████████████████| 80 kB 3.8 MB/s 
     |████████████████████████████████| 68 kB 5.7 MB/s 
     |████████████████████████████████| 46 kB 1.8 MB/s 
     |████████████████████████████████| 4.1 MB 36.3 MB/s 
     |████████████████████████████████| 856 kB 55

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/MyDrive/Models/Anxiety_Dict.p', 'rb') as handle:
    anxiety_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/Control_Dict.p', 'rb') as handle:
    control_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/Depression_Dict.p', 'rb') as handle:
    depression_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/Autism_Dict.p', 'rb') as handle:
    autism_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/BPD_Dict.p', 'rb') as handle:
    BPD_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/Bipolar_Dict.p', 'rb') as handle:
    bipolar_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/Schizo_Dict.p', 'rb') as handle:
    schizo_word_index = pickle.load(handle)


anxiety_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Anxiety')
control_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Control')
depression_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Depression')
autism_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Autism')
BPD_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_BPD')
bipolar_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Bipolar')
schizo_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Schizo')

In [5]:
def detokenize(tokens):
    text = ' '.join(tokens)
    return text

def preprocess_corpus(tokens):
  tokens = [t.lower() for t in tokens] # make lowercase

  stop = set(stopwords.words('english'))
  tokens = [t for t in tokens if t not in stop] # remove stop words
  tokens = list(filter(None, tokens)) # get rid of extra spaces

  text = detokenize(tokens)


  #text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text) #put spaces between word and special characters
  #text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", text) #put spaces between special characters and word
  #text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"([.,;:!?'\"“\(])(\w)", text) #put spaces between special characters and other special characters

  pattern = r"[{}]".format(string.punctuation) 
  text = re.sub(pattern, "", text)  # remove puntucation 
  text = re.sub(r'\b\d+\b', '', text) # only accept text
  
  tokens = re.split(r"\s+",text) #split the words into array
  tokens = list(filter(None, tokens)) # get rid of extra spaces
  
  return tokens

def getProcessCorpus(data):
  processed_data = []
  for i in range(len(data)):
    sentence = data[i]
    p_sentence = preprocess_corpus(sentence)
    processed_data.append(p_sentence[1:len(p_sentence)])
  return processed_data

In [6]:
def runMentalHealthAlarmSystem(input_sentence):
  processed_sentence = preprocess_corpus(input_sentence.split(' '))

  x_anxiety = []
  x_control = []
  x_depression = []
  x_autism = []
  x_BPD = []
  x_bipolar = []
  x_schizo = []

  x_anxiety_numeric = []
  x_control_numeric = []
  x_depression_numeric = []
  x_autism_numeric = []
  x_BPD_numeric = []
  x_bipolar_numeric = []
  x_schizo_numeric = []

  for token in processed_sentence:
    x_anxiety_numeric.append(anxiety_word_index[token])
    x_control_numeric.append(control_word_index[token])
    x_depression_numeric.append(depression_word_index[token])
    x_autism_numeric.append(autism_word_index[token])
    x_BPD_numeric.append(BPD_word_index[token])
    x_bipolar_numeric.append(bipolar_word_index[token])
    x_schizo_numeric.append(schizo_word_index[token])

  x_anxiety.append(x_anxiety_numeric)
  x_control.append(x_control_numeric)
  x_depression.append(x_depression_numeric)
  x_autism.append(x_autism_numeric)
  x_BPD.append(x_BPD_numeric)
  x_bipolar.append(x_bipolar_numeric)
  x_schizo.append(x_schizo_numeric)

  max_sequence_length = 5000

  x_anxiety = np.array(pad_sequences(x_anxiety, maxlen = 4792, padding = 'pre'))
  x_control = np.array(pad_sequences(x_control, maxlen = 4792, padding = 'pre'))
  x_depression = np.array(pad_sequences(x_depression, maxlen = 8403, padding = 'pre'))
  x_autism = np.array(pad_sequences(x_autism, maxlen = 3902, padding = 'pre'))
  x_BPD = np.array(pad_sequences(x_BPD, maxlen = 2996, padding = 'pre'))
  x_bipolar = np.array(pad_sequences(x_bipolar, maxlen = 4792, padding = 'pre'))
  x_schizo = np.array(pad_sequences(x_schizo, maxlen = 2781, padding = 'pre'))

  mental_health_stats = []

  anxiety_percentage = (anxiety_model.predict(x_anxiety))
  control_percentage = (control_model.predict(x_control))
  depression_percentage = (depression_model.predict(x_depression))
  autism_percentage = (autism_model.predict(x_autism))
  BPD_percentage = (BPD_model.predict(x_BPD))
  bipolar_percentage = (bipolar_model.predict(x_bipolar))
  schizo_percentage = (schizo_model.predict(x_schizo))

  mental_health_stats.append("{:.2%}".format(anxiety_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(control_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(depression_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(autism_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(BPD_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(bipolar_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(schizo_percentage[0][1]))

  df = pd.DataFrame(mental_health_stats, columns = ['Percentage Chance of Disorder'])
  df['Disorder'] = ['Anxiety', 'Control', 'Depression', 'Autism', 'Borderline Personality', 'Bipolar', 'Schizo']
  df['Description'] = ['Assess similarity to Reddit discussion of Anxiety', 
                       'Assess similarity to Reddit forum to discuss, vent, support and share information about mental health, illness, and wellness.', 
                       'Assess similarity to Reddit discussion of Depression',
                       'Assess similarity to Reddit discussion of Autism',
                       'Assess similarity to Reddit discussion of Borderline Personality Disorder',
                       'Assess similarity to Reddit discussion of Bipolar',
                       'Assess similarity to Reddit disccusion of Schizo']
  
  return df


In [7]:
sentence = 'I am depressed and upset, and I dont know what to do'
df = runMentalHealthAlarmSystem(sentence)
print(df)

1/1 [==============================] - 0s 83ms/step


1/1 [==============================] - 0s 93ms/step


1/1 [==============================] - 0s 78ms/step
  Percentage Chance of Disorder                Disorder  \
0                         0.72%                 Anxiety   
1                        34.18%                 Control   
2                        96.03%              Depression   
3                        38.39%                  Autism   
4                        28.15%  Borderline Personality   
5                        39.26%                 Bipolar   
6                         2.83%                  Schizo   

                                         Description  
0  Assess similarity to Reddit discussion of Anxiety  
1  Assess similarity to Reddit forum to discuss, ...  
2  Assess similarity to Reddit discussion of Depr...  
3   Assess similarity to Reddit discussion of Autism  
4  Assess similarity to Reddit discussion of Bord...  
5  Assess similarity to Reddit discussion of Bipolar  
6   Assess similarity to Reddit disccusion of Schizo  


In [9]:
iface = gr.Interface(fn=runMentalHealthAlarmSystem, inputs="text", outputs="dataframe")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://67e8b143a7982bf5.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7facf7840cd0>,
 'http://127.0.0.1:7861/',
 'https://67e8b143a7982bf5.gradio.app')